<a href="https://colab.research.google.com/github/26medias/TF-Face-Angle-Translation/blob/master/Face_Position_Dataset_Builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Angle Dataset Generator

In [0]:
CAPTURE_FPS = 1 #@param {type:"slider", min:1, max:30, step:1}

import requests
import ntpath
import cv2
import os, sys

# The variables
DIR_VIDEOS = "videos"
DIR_IMAGES = "images"
#CAPTURE_FTP  = 2 # We'll extract 2 images per second of video

if not os.path.isdir(DIR_VIDEOS):
  os.mkdir(DIR_VIDEOS, 755);
if not os.path.isdir(DIR_IMAGES):
  os.mkdir(DIR_IMAGES, 755);

def downloadFile(url):
  myfile = requests.get(url)
  filename = ntpath.basename(url)
  open(filename, 'wb').write(myfile.content)
  return filename


def videoToImages(filename, capture_fps=1):
  basename = os.path.splitext(filename)[0]
  os.mkdir(basename, 755);
  cap = cv2.VideoCapture(filename)
  # Get the video's FPS
  fps = cap.get(cv2.CAP_PROP_FPS)
  # How many frames between capture?
  skipFrame = round(fps/capture_fps)
  print(basename, ": ftp: ",ftp," / skipFrame: ", skipFrame)
  i = 0
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      if (i % skipFrame == 0):
        continue
      cv2.imwrite(basename+'/'+str(i)+'.jpg',frame)
      i+=1
  cap.release()
  cv2.destroyAllWindows()
  
  
#videoFilename = downloadFile("http://trailers.apple.com/movies/fox_searchlight/lucy-in-the-sky/lucy-in-the-sky-trailer-1_h480p.mov")
#videoToImages(videoFilename)